In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import time
from keras.applications import vgg16
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential,load_model
from keras.utils import to_categorical
import keras
from keras.layers import InputLayer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from keras.layers import InputLayer

Using TensorFlow backend.


In [2]:
data_path = '/home/vishwajeet/Documents/Mask_detector/mask_dataset/'
data_dir_list = os.listdir(data_path)

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.resize(input_img,(224,224))
        img_data_list.append(input_img)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-without_mask

Loaded the images of dataset-with_mask



(3833, 224, 224, 3)

In [3]:
num_classes = 2

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:1918]=0 #1918
labels[1918:3833]=1 #1915

labels =to_categorical(labels, num_classes)
x_train, x_test, y_train, y_test = train_test_split(img_data, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    stratify=np.array(labels), 
                                                    random_state=4)
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_CHANNELS=3
input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS)
x_train=x_train.reshape(x_train.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
x_test=x_test.reshape(x_test.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
# x_train.shape,x_test.shape
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [4]:
input_shape=(224,224,3)
vgg = vgg16.VGG16(include_top=True, weights='imagenet')

out = Dense(2, activation='softmax', name='predictions')(vgg.layers[-2].output)
vgg_model = Model(vgg.input, out)

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['predictions']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x7fed70fbf3d0>,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0x7fed9750df10>,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0x7fed70f95c90>,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0x7fedbd1c31d0>,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0x7fedc0b92e50>,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0x7fece4379850>,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0x7fed70fbfed0>,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0x7fece438b990>,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0x7fece43918d0>,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0x7fece4391ad0>,block3_conv3,False


In [5]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
vgg_model.compile(Adam(lr=.001), loss='binary_crossentropy', metrics=['accuracy']) 
BATCH_SIZE=30
epochs=10

vgg_model.fit(x_train, y_train,epochs=epochs, batch_size=BATCH_SIZE)
# history=vgg_model.fit(x_train, y_train, epochs=50, batch_size=30,verbose=1)

Epoch 1/10
3066/3066 [==============================] - 508s 166ms/step - loss: 0.4179 - accuracy: 0.8190
Epoch 2/10
3066/3066 [==============================] - 549s 179ms/step - loss: 0.2550 - accuracy: 0.9083
Epoch 3/10
3066/3066 [==============================] - 520s 170ms/step - loss: 0.2079 - accuracy: 0.9263
Epoch 4/10
3066/3066 [==============================] - 468s 153ms/step - loss: 0.2052 - accuracy: 0.9188
Epoch 5/10
3066/3066 [==============================] - 495s 161ms/step - loss: 0.1773 - accuracy: 0.9282
Epoch 6/10
3066/3066 [==============================] - 469s 153ms/step - loss: 0.1511 - accuracy: 0.9488
Epoch 7/10
3066/3066 [==============================] - 469s 153ms/step - loss: 0.1399 - accuracy: 0.9504
Epoch 8/10
3066/3066 [==============================] - 467s 152ms/step - loss: 0.1325 - accuracy: 0.9543
Epoch 9/10
3066/3066 [==============================] - 469s 153ms/step - loss: 0.1351 - accuracy: 0.9508
Epoch 10/10
3066/3066 [=======================

In [7]:
vgg_model.save('md_model.h5')